<a href="https://colab.research.google.com/github/yeonghwanchoi/EDA/blob/master/jeju_EDA/sale_trend_jeju.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
python3-gdal is already the newest version (2.2.3+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-rtree is already the newest version (0.8.3+ds-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-9o9z96pb
  Running command git clone -q git://github.com/geopandas/geopandas.git /tmp/pip-req-build-9o9z96pb
  Created wheel for geopandas: filename=geopandas-0.9.0+36.gcb88dd4-py2.py3-none-any.whl size=996489 sha256=9b4d3d2ead143858d275928630264b04c3366a110ee93ed81c1d609dfe72a8f9
  Stored in directory: /tmp/pip-ephem-wheel-cache-94fgyutj/wheels/91/24/71/376c9c67192694168352

# EDA : sale trend in jeju island 

1.data preprocessing 


In [3]:
#data load

from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

plt.rc('font', family='AppleGothic')

import folium 
import geopandas as gpd
from IPython.display import Image

import tqdm # progress bar 
import os, sys

from scipy.spatial import distance
from sklearn.cluster import AgglomerativeClustering

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
path = '/content/drive/MyDrive/colab/jeju/data/'

data5= pd.read_csv(os.path.join(path,'KRI-DAC_Jeju_data5.txt'))
data6= pd.read_csv(os.path.join(path,'KRI-DAC_Jeju_data6.txt'))
data7= pd.read_csv(os.path.join(path,'KRI-DAC_Jeju_data7.txt'))
data8= pd.read_csv(os.path.join(path,'KRI-DAC_Jeju_data8.txt'))

df = pd.concat([data5, data6, data7, data8]).drop(['X','Y'], axis=1).reset_index(drop=True)
df.columns=df.columns.str.lower()

In [5]:
# df.describe()

In [6]:
#data preprocessing

In [291]:
# geocoding

# 
path = '/content/drive/MyDrive/colab/jeju/data/'

data5= pd.read_csv(os.path.join(path,'KRI-DAC_Jeju_data5.txt'))
data6= pd.read_csv(os.path.join(path,'KRI-DAC_Jeju_data6.txt'))
data7= pd.read_csv(os.path.join(path,'KRI-DAC_Jeju_data7.txt'))
data8= pd.read_csv(os.path.join(path,'KRI-DAC_Jeju_data8.txt'))

df = pd.concat([data5, data6, data7, data8]).drop(['X','Y'], axis=1).reset_index(drop=True)
df.columns=df.columns.str.lower()



from pyproj import Proj, transform 
proj_itrf = Proj(init='epsg:5178') # korean grid
proj_wgs84 = Proj(init='epsg:4326') # World Geodetic System

df["lon"],df["lat"] = transform(proj_itrf,proj_wgs84,df['point_x'],df['point_y']) # itrf --> lon/lat
df['geometry'] = gpd.points_from_xy(df.lon, df.lat) # points 

boundary = gpd.read_file(os.path.join(path, 'LSMD_ADM_SECT_UMD_50.shp'),crs='EPSG:5178',encoding='euc-kr').to_crs('EPSG:4326') # load data (WGS)


In [292]:
df.head()

,objectid,field1,ym,sido,sigungu,franclass,type,time,totalspent,disspent,numofspent,numofdisspent,point_x,point_y,lon,lat,geometry
0,1,1,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,877005.9834,1.479766e+06,126.176746,33.305090,POINT (126.17675 33.30509)
1,2,2,202005,제주특별자치도,제주시,영세,단란주점,00시,1180000,0,3,0,877005.7447,1.479816e+06,126.176737,33.305540,POINT (126.17674 33.30554)
2,3,3,202005,제주특별자치도,제주시,중소1,편의점,00시,157670,6850,20,2,877056.6756,1.479616e+06,126.177311,33.303746,POINT (126.17731 33.30375)
3,4,4,202005,제주특별자치도,제주시,영세,편의점,00시,46600,0,2,0,877055.9593,1.479766e+06,126.177283,33.305098,POINT (126.17728 33.30510)
4,5,5,202005,제주특별자치도,제주시,영세,주점,00시,66000,0,2,0,877055.4817,1.479866e+06,126.177264,33.305999,POINT (126.17726 33.30600)


In [293]:
def towns_boundary(boundary, df): 
    coor_df = gpd.GeoDataFrame(columns={'town', 'geometry'}) # dataframe for coordinates 
    coors_list = df['geometry'].unique() 
    
    for num in range(len(boundary)):
 
        town_boundary_code= boundary.loc[num, 'geometry'] 
        town_name = boundary.loc[num, 'EMD_NM']
        result = coors_list.within(town_boundary_code)

        idx = [num for num, i in enumerate(result) if i]
        coor_df = coor_df.append(gpd.GeoDataFrame({'town':[town_name for _ in range(len(idx))],'geometry':coors_list[idx]}))

    coor_df = coor_df.drop_duplicates(subset='geometry', keep='first').reset_index(drop=True) #just in case before merge
    df = df.merge(coor_df, how='left', on='geometry') 

    #if there is null data
    null_data= df[df['town'].isnull()]['geometry']
    coor_null = null_data.unique()
    for code in coor_null:
        idx = coor_df['geometry'].distance(code).argmin() # find a point near 
        town_name = coor_df.loc[idx, 'town']
        coor_df = coor_df.append(gpd.GeoDataFrame({'town':[town_name], 'geometry': [code]}), ignore_index=True)
        
    df.drop(columns='town', inplace=True)
    df = df.merge(coor_df, how='left', on='geometry')
    return df, null_data    

df, null_df = towns_boundary(boundary, df)
df

,objectid,field1,ym,sido,sigungu,franclass,type,time,totalspent,disspent,numofspent,numofdisspent,point_x,point_y,lon,lat,geometry,town
0,1,1,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,877005.9834,1.479766e+06,126.176746,33.305090,POINT (126.17675 33.30509),한경면
1,2,2,202005,제주특별자치도,제주시,영세,단란주점,00시,1180000,0,3,0,877005.7447,1.479816e+06,126.176737,33.305540,POINT (126.17674 33.30554),한경면
2,3,3,202005,제주특별자치도,제주시,중소1,편의점,00시,157670,6850,20,2,877056.6756,1.479616e+06,126.177311,33.303746,POINT (126.17731 33.30375),한경면
3,4,4,202005,제주특별자치도,제주시,영세,편의점,00시,46600,0,2,0,877055.9593,1.479766e+06,126.177283,33.305098,POINT (126.17728 33.30510),한경면
4,5,5,202005,제주특별자치도,제주시,영세,주점,00시,66000,0,2,0,877055.4817,1.479866e+06,126.177264,33.305999,POINT (126.17726 33.30600),한경면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119424,280081,280081,202008,제주특별자치도,제주시,영세,편의점,x시,13700,0,2,0,950266.5976,1.502458e+06,126.962369,33.515632,POINT (126.96237 33.51563),우도면
1119425,280082,280082,202008,제주특별자치도,제주시,영세,일반한식,x시,56000,0,1,0,950317.5379,1.502258e+06,126.962928,33.513833,POINT (126.96293 33.51383),우도면
1119426,280083,280083,202008,제주특별자치도,제주시,영세,일반한식,x시,28000,0,1,0,950469.1585,1.501909e+06,126.964580,33.510691,POINT (126.96458 33.51069),우도면
1119427,280084,280084,202008,제주특별자치도,제주시,중소1,중국음식,x시,46000,0,3,0,950574.6388,1.500760e+06,126.965779,33.500332,POINT (126.96578 33.50033),우도면


In [294]:
null_df.to_csv('null값.csv', index=True)
df.to_csv('data.csv', index=False)

In [271]:
def towns_boundary(boundary, df): 
    coor_df = gpd.GeoDataFrame(columns={'town', 'geometry'}) # dataframe for coordinates 
    coors_list = df['geometry'].unique() 
    
    for num in range(len(boundary)):
 
        town_boundary_code= boundary.loc[num, 'geometry'] 
        town_name = boundary.loc[num, 'EMD_NM']
        result = coors_list.within(town_boundary_code)

        idx = [num for num, i in enumerate(result) if i]
        coor_df = coor_df.append(gpd.GeoDataFrame({'town':[town_name for _ in range(len(idx))],'geometry':coors_list[idx]}))

    coor_df = coor_df.drop_duplicates(subset='geometry', keep='first').reset_index(drop=True) 
    df = df.merge(coor_df, how='left', on='geometry') 

    null_df = df.loc[df['town'].isnull(), 'geometry'] 
    print("Null data : {}".format(len(null_df)))
    
    # 포함되지 않는 좌표는 가장 가까운 점의 동으로 설정
    coor_null = null_df.unique()
    for xy in tqdm.tqdm_notebook(coor_null):
        idx = coor_df['geometry'].distance(xy).argmin()
        dong_name = coor_df.loc[idx, 'town']
        coor_df = coor_df.append(gpd.GeoDataFrame({'town':[dong_name], 'geometry': [xy]}), ignore_index=True)

    df.drop(columns='town', inplace=True)
    df = df.merge(coor_df, how='left', on='geometry')
    return df, null_df

In [272]:
data,null_df=towns_boundary(boundary, df) 

Null data : 101527


In [ ]:
# import sys
# from data import api_key 
# key=api_key.google_api_key

In [ ]:
df=pd.concat(concat_list)

In [ ]:
from pyproj import Proj, transform 
proj_itrf = Proj(init='epsg:5178')
proj_wgs84 = Proj(init='epsg:4326')
df["lon"],df["lat"] = transform(proj_itrf,proj_wgs84,df['point_x'],df["point_y"])
df.drop(['point_x','point_y',"field1",'ym'],inplace=True, axis=1)

In [ ]:
# df['lon_lat']=df['lon'].astype(str)+'_'+df['lat'].astype(str)

In [ ]:
# df_lon_lat=df['lon_lat'].drop_duplicates()
# df_lon_lat

In [ ]:
# import googlemaps
# from datetime import datetime

# gmaps = googlemaps.Client(key) 
# reverse_geocode_result = gmaps.reverse_geocode((33.302315,126.176746))
# reverse_geocode_result
# # geocode_result  

In [ ]:
# import json
# m=folium.Map(location=[48,-102],zoom_start=3)
# folium.Choropleth(
#     geo_data='./data/03_us-states.json',
#     data=state_data, columns=["State",'Unemployment'],
#     nan_fill_color="purple", nan_fill_opacity=0.4,
#     key_on="    ", fill_color="YlGn",
#     legend_name="Unemployment Rate (%)"
# ).add_to(m)
# m

In [ ]:
#import rename module  
import type_names
input_data=type_names.rename()
for key, value in input_data:
    df["type"].replace(dict.fromkeys(key, value), inplace=True)

In [ ]:
df

In [ ]:
# data preprocessing

df['sigungu']=df['sigungu'].replace(['제주시','서귀포시'],['jeju','Seogwipo'])
df=df[df['time']!='x시'].reset_index(drop=True) # payments canceled 
df['time']=df['time'].apply(lambda x : x[:2])
df.franclass=df.franclass.replace(['영세', '중소1', '중소', '중소2', '일반'],['under_0.3m','under_1m','under_0.5m','under_3m','over_3m'])
df_final=df.rename(columns={'sigungu':'city','franclass':'class','totalspent':'total_income','disspent':'subsidies_income','numofspent':'num_of_payments','numofdisspent':'num_of_subspayment'})
df_final[['total_income','subsidies_income']]=df_final[['total_income','subsidies_income']].astype(int).apply(lambda x:x/1000) #won to $, 1000:1
df_final['non_subsidies_income']=df_final['total_income']-df_final['subsidies_income']

In [ ]:
df_final['month'].unique()

In [ ]:
# visualization 
#1. subsidies_income by month 
df_month_group=df_final.groupby('month')["subsidies_income"].sum().reset_index()

plt.figure(figsize=(10,6))
sns.catplot(x="month", y="subsidies_income", ci=None, kind="bar", data=df_month_group);

In [ ]:
df_final

In [ ]:
# all income by city 
# population = jeju_city (507,095), seogwipo (190,254), seogwipo/jeju_city = 0.375
df_city_group= df_final.groupby('city')['total_income','non_subsidies_income',"subsidies_income"].sum() 
df_city_group

f, ax = plt.subplots(1,3, figsize=(20,10))

df_city_group['total_income'].plot.pie(explode=[0,0.1], autopct='%1.1f%%',ax=ax[0], shadow=True,fontsize=20)
ax[0].set_title('total income by city',fontsize=20)

df_city_group['non_subsidies_income'].plot.pie(explode=[0,0.1], autopct='%1.1f%%',ax=ax[1], shadow=True,fontsize=20)
ax[1].set_title('non_subsidies_income by city',fontsize=20)

df_city_group['subsidies_income'].plot.pie(explode=[0,0.1], autopct='%1.1f%%',ax=ax[2], shadow=True, fontsize=20)
ax[2].set_title('subsidies_income by city',fontsize=20)


In [ ]:
#
df_5=df_final[df_final['month']=='5']
df_5_sum=df_5[['non_subsidies_income','subsidies_income']].sum()

plt.figure(figsize=(10,6))
df_5_sum_plot=df_5_sum.plot.pie(explode=[0,0.1], autopct='%1.1f%%', shadow=True,fontsize=20)


In [ ]:
df_5_group_type=df_5.groupby('type')['total_income','non_subsidies_income',"subsidies_income"].sum() 
df_5_group_type.iloc[1]

total_income            1.292806e+07
non_subsidies_income    1.091423e+07
subsidies_income        2.013829e+06
Name: automobile, dtype: float64

In [ ]:
#error fix
df_5_group_type.loc['beauty']=df_5_group_type.loc['beauty']+df_5_group_type.loc['화장품']
df_5_group_type.drop(index='화장품', inplace=True)

In [ ]:
df_5_group_type['replacement_rate']=df_5_group_type['subsidies_income']/df_5_group_type['non_subsidies_income']
df_5_group_type.sort_values('replacement_rate',ascending=False)

,total_income,non_subsidies_income,subsidies_income,replacement_rate
type,,,,
agriculture,6.499023e+06,4.669299e+06,1829724.194,0.391863
grocery,4.010482e+07,3.236285e+07,7741962.050,0.239224
housing,1.386279e+06,1.147461e+06,238817.841,0.208127
beauty,1.099585e+06,9.206987e+05,178886.697,0.194295
construction,4.322700e+06,3.621635e+06,701065.099,0.193577
automobile,1.292806e+07,1.091423e+07,2013829.169,0.184514
medical,1.110183e+07,9.426260e+06,1675572.420,0.177756
dinning,3.804076e+07,3.264970e+07,5391056.740,0.165118
service,4.397064e+06,3.777110e+06,619954.400,0.164135


In [ ]:
import itertools
a=list(itertools.permutations('ABCD', 2))
print(a)

[('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'A'), ('B', 'C'), ('B', 'D'), ('C', 'A'), ('C', 'B'), ('C', 'D'), ('D', 'A'), ('D', 'B'), ('D', 'C')]


In [ ]:
df = gpd.read_file(gpd.datasets.get_path('nybb'))

In [ ]:
df

,BoroCode,BoroName,Shape_Leng,Shape_Area,geometry
0,5,Staten Island,330470.010332,1.623820e+09,"MULTIPOLYGON (((970217.022 145643.332, 970227...."
1,4,Queens,896344.047763,3.045213e+09,"MULTIPOLYGON (((1029606.077 156073.814, 102957..."
2,3,Brooklyn,741080.523166,1.937479e+09,"MULTIPOLYGON (((1021176.479 151374.797, 102100..."
3,1,Manhattan,359299.096471,6.364715e+08,"MULTIPOLYGON (((981219.056 188655.316, 980940...."
4,2,Bronx,464392.991824,1.186925e+09,"MULTIPOLYGON (((1012821.806 229228.265, 101278..."


In [ ]:
import geopandas

path_to_data = geopandas.datasets.get_path("nybb")
gdf = geopandas.read_file(path_to_data)

gdf

,BoroCode,BoroName,Shape_Leng,Shape_Area,geometry
0,5,Staten Island,330470.010332,1.623820e+09,"MULTIPOLYGON (((970217.022 145643.332, 970227...."
1,4,Queens,896344.047763,3.045213e+09,"MULTIPOLYGON (((1029606.077 156073.814, 102957..."
2,3,Brooklyn,741080.523166,1.937479e+09,"MULTIPOLYGON (((1021176.479 151374.797, 102100..."
3,1,Manhattan,359299.096471,6.364715e+08,"MULTIPOLYGON (((981219.056 188655.316, 980940...."
4,2,Bronx,464392.991824,1.186925e+09,"MULTIPOLYGON (((1012821.806 229228.265, 101278..."


In [ ]:
gdf = gdf.set_index("BoroName")

In [ ]:
gdf

,BoroCode,Shape_Leng,Shape_Area,geometry
BoroName,,,,
Staten Island,5,330470.010332,1.623820e+09,"MULTIPOLYGON (((970217.022 145643.332, 970227...."
Queens,4,896344.047763,3.045213e+09,"MULTIPOLYGON (((1029606.077 156073.814, 102957..."
Brooklyn,3,741080.523166,1.937479e+09,"MULTIPOLYGON (((1021176.479 151374.797, 102100..."
Manhattan,1,359299.096471,6.364715e+08,"MULTIPOLYGON (((981219.056 188655.316, 980940...."
Bronx,2,464392.991824,1.186925e+09,"MULTIPOLYGON (((1012821.806 229228.265, 101278..."


In [ ]:
a=gdf.boundary

In [ ]:
a

BoroName
Staten Island    MULTILINESTRING ((970217.022 145643.332, 97022...
Queens           MULTILINESTRING ((1029606.077 156073.814, 1029...
Brooklyn         MULTILINESTRING ((1021176.479 151374.797, 1021...
Manhattan        MULTILINESTRING ((981219.056 188655.316, 98094...
Bronx            MULTILINESTRING ((1012821.806 229228.265, 1012...
dtype: geometry